In [12]:
from selenium.webdriver import Chrome
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By

In [13]:
driver = Chrome()
driver.get('https://twitter.com/i/flow/login')

In [14]:
username = driver.find_element(By.XPATH,'//input[@name = "text"]')
username.send_keys('nad.yul2002@gmail.com')

In [15]:
username.send_keys(Keys.RETURN)

In [16]:
my_password = getpass()

········


In [17]:
password = driver.find_element(By.XPATH, '//input[@name = "password"]')
password.send_keys(my_password)

In [18]:
password.send_keys(Keys.RETURN)

In [19]:
search_input = driver.find_element(By.XPATH, '//input[@aria-label="Search query"]')
search_input.send_keys('The New York Times')
search_input.send_keys(Keys.RETURN)

In [24]:
driver.find_element_by_link_text('People').click()
nytimes_input = driver.find_element(By.XPATH, '//span[text() = "The New York Times"]')
nytimes_input.click()

<ipython-input-24-4beab4331ad6>:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_link_text('People').click()


In [25]:
cards = driver.find_elements(By.XPATH, './/article[@data-testid="tweet"]')
card = cards[0]

In [26]:
#Tweet Publisher
card.find_element(By.XPATH, './/span').text

'The New York Times'

In [121]:
#Tweet Date
card.find_element(By.XPATH, './/time').get_attribute('datetime')

'2021-12-07T14:50:04.000Z'

In [122]:
#Tweet Content 
comment = card.find_element(By.XPATH, './/div[1]/div[1]/div[1]/div[2]/div[2]/div[2]').text
responding = card.find_element(By.XPATH, './/div[1]/div[1]/div[2]').text
comment+responding

'Here are the best mystery novels of 2021, according to our crime fiction columnist.\nnytimes.com\nThe Best Mystery Novels of 2021\nOur crime fiction columnist picks the books that wowed her this year.\n5\n28\n144The New York Times\n@nytimes\n·\n9m\nHere are the best mystery novels of 2021, according to our crime fiction columnist.\nnytimes.com\nThe Best Mystery Novels of 2021\nOur crime fiction columnist picks the books that wowed her this year.\n5\n28\n144'

In [123]:
#Tweet Reply Count
card.find_element(By.XPATH, './/div[@data-testid="reply"]').text

'5'

In [124]:
#Tweet Retweet
card.find_element(By.XPATH, './/div[@data-testid="retweet"]').text

'28'

In [125]:
#Tweet Likes
card.find_element(By.XPATH, './/div[@data-testid="like"]').text

'144'

In [126]:
def get_tweet_data(card):
    """Extract data from tweet"""
    username = card.find_element(By.XPATH, './/span').text
    handle = card.find_element(By.XPATH, './/span[contains(text(), "@")]').text
    try:
        postdate = card.find_element(By.XPATH, './/time').get_attribute('datetime')
    except NoSuchElementException:
        return
    comment = card.find_element(By.XPATH, './/div[1]/div[1]/div[1]/div[2]/div[2]/div[2]').text
    responding = card.find_element(By.XPATH, './/div[1]/div[1]/div[2]').text
    text = comment+responding
    reply_cnt = card.find_element(By.XPATH, './/div[@data-testid="reply"]').text
    retweet_cnt = card.find_element(By.XPATH, './/div[@data-testid="retweet"]').text
    likes_cnt = card.find_element(By.XPATH, './/div[@data-testid="like"]').text
    
    tweet = (username, handle, postdate, text, reply_cnt, retweet_cnt, likes_cnt)
    return tweet

In [127]:
get_tweet_data(card)

('The New York Times',
 '@nytimes',
 '2021-12-07T14:50:04.000Z',
 'Here are the best mystery novels of 2021, according to our crime fiction columnist.\nnytimes.com\nThe Best Mystery Novels of 2021\nOur crime fiction columnist picks the books that wowed her this year.\n5\n28\n144The New York Times\n@nytimes\n·\n9m\nHere are the best mystery novels of 2021, according to our crime fiction columnist.\nnytimes.com\nThe Best Mystery Novels of 2021\nOur crime fiction columnist picks the books that wowed her this year.\n5\n28\n144',
 '5',
 '28',
 '144')

In [128]:
tweet_data = []

for card in cards:
    data = get_tweet_data(card)
    if data:
        tweet_data.append(data)

tweet_data

[('The New York Times',
  '@nytimes',
  '2021-12-07T14:50:04.000Z',
  'Here are the best mystery novels of 2021, according to our crime fiction columnist.\nnytimes.com\nThe Best Mystery Novels of 2021\nOur crime fiction columnist picks the books that wowed her this year.\n5\n28\n147The New York Times\n@nytimes\n·\n9m\nHere are the best mystery novels of 2021, according to our crime fiction columnist.\nnytimes.com\nThe Best Mystery Novels of 2021\nOur crime fiction columnist picks the books that wowed her this year.\n5\n28\n147',
  '5',
  '28',
  '147'),
 ('The New York Times Retweeted',
  '@EricLiptonNYT',
  '2021-12-07T12:41:23.000Z',
  'PART IV of NYT Series: Ambassadors, mercenaries, celebrities, musicians, entrepreneurs — they all pass through the 5-star Fleuve Congo Hotel in Kinshasa, where deals are made for Congo’s abundant minerals like cobalt & copper. \n@dionnesearcey\n @PekingMike\nnytimes.com\nOn the Banks of the Furious Congo River, a 5-Star Emporium of Ambition\nAs the cl

In [129]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')

In [130]:
#get all tweets on the page

data = []
tweet_ids = set()
last_position = driver.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    page_cards = driver.find_elements(By.XPATH, './/article[@data-testid="tweet"]')
    for card in page_cards[-15:]:
        tweet = get_tweet_data(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
    
    scroll_attempt = 0
    while True:            
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(1)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1

            if scroll_attempt >=3:
                scrolling = False
                break
            else:
                sleep(2)
        else:
            last_position = curr_position
            break

In [131]:
data

[('The New York Times',
  '@nytimes',
  '2021-12-07T12:00:14.000Z',
  'A 73-year-old volunteer died on Saturday after she was repeatedly rammed by a sheep while working at a Massachusetts farm that uses animals in mental health therapy, the authorities said.\nnytimes.com\nVolunteer Dies After a Sheep Charges at Her on a Therapy Farm\nKim Taylor, 73, went into cardiac arrest after being attacked while caring for livestock at a Massachusetts farm, the police said.\n41\n71\n224The New York Times\n@nytimes\n·\n3h\nA 73-year-old volunteer died on Saturday after she was repeatedly rammed by a sheep while working at a Massachusetts farm that uses animals in mental health therapy, the authorities said.\nnytimes.com\nVolunteer Dies After a Sheep Charges at Her on a Therapy Farm\nKim Taylor, 73, went into cardiac arrest after being attacked while caring for livestock at a Massachusetts farm, the police said.\n41\n71\n224',
  '41',
  '71',
  '224'),
 ('The New York Times',
  '@nytimes',
  '2021-1

In [134]:
with open('NYT_tweets.csv', 'w', newline = '', encoding = 'utf-8') as f:
    header = ['UserName', 'Handle', 'Timestamp', 'Comments', 'Likes', 'Retweets', 'Text']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)